#balancing failure vs success

In [1]:
from pathlib import Path
# Set working directory and list files to confirm CSV presence
cwd = Path('.').resolve()
print('Working dir:', cwd)
print('Files in notebook dir:')
for p in sorted(Path('.').glob('*'))[:50]:
    print('-', p)


Working dir: /work/shibberu/share/MA384_Data_Mining_Projects_Winter_2025-26/SMART_failure_prediction/S.M.A.R.T/example_data/example_Q3_2025
Files in notebook dir:
- 2025-07-01_reduced (balance test).csv
- 2025-07-01_reduced.csv
- Balancing test.ipynb
- Nan values test.ipynb
- balance test (2025-07-01).ipynb
- balanced_clean_dataset.csv
- balanced_dataset_final.csv
- cutdown_csv_for_github.py


In [2]:
# Install/import libraries and display options
import pandas as pd
from pathlib import Path
pd.set_option('display.max_columns', 200)


In [3]:
# Path to the CSV we found in the repo
csv_path = Path('example_data/example_Q3_2025/2025-07-01_reduced (balance test).csv')
print('CSV exists:', csv_path.exists())
print('CSV absolute path:', csv_path.resolve())

# Load CSV (adjust encoding/low_memory as needed)
if csv_path.exists():
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
else:
    df = pd.DataFrame()
    print('CSV not found at the expected location; update `csv_path` to the correct file.')


CSV exists: False
CSV absolute path: /work/shibberu/share/MA384_Data_Mining_Projects_Winter_2025-26/SMART_failure_prediction/S.M.A.R.T/example_data/example_Q3_2025/example_data/example_Q3_2025/2025-07-01_reduced (balance test).csv
CSV not found at the expected location; update `csv_path` to the correct file.


In [4]:
# Preview dataset
if not df.empty:
    print('Shape:', df.shape)
    display(df.head())
    print('\nColumns:')
    print(list(df.columns))
    print('\nData types:')
    display(df.dtypes)
    display(df.sample(5))
else:
    print('DataFrame is empty.\n')


DataFrame is empty.



In [5]:
# Normalize column names
if not df.empty:
    df.columns = (
        df.columns
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
        .str.replace('[^0-9a-z_]', '', regex=True)
    )
    print('Normalized columns:')
    print(list(df.columns)[:50])


In [6]:
# Missing values summary
if not df.empty:
    missing = df.isna().sum().sort_values(ascending=False)
    display(missing[missing>0].head(50))


In [7]:
# Convert numeric-like columns where possible
if not df.empty:
    for col in df.columns:
        # try to coerce numeric values
        df[col] = pd.to_numeric(df[col], errors='ignore')
    print('Attempted numeric conversion on object columns.')


In [8]:
# Check for common target columns and show class balance if present
possible_targets = ['failure', 'failed', 'target', 'label', 'y', 'is_failed']
found_targets = [c for c in df.columns if c in possible_targets] if not df.empty else []
if found_targets:
    for t in found_targets:
        print('\nTarget column:', t)
        print(df[t].value_counts(dropna=False))
        print(df[t].value_counts(normalize=True, dropna=False))
else:
    print('No common target column names found. Inspect `df.columns` to identify the target.')


No common target column names found. Inspect `df.columns` to identify the target.


In [9]:
# Save cleaned dataframe (if loaded)
if not df.empty:
    out = Path('2025-07-01_reduced_balance_test_cleaned.csv')
    df.to_csv(out, index=False)
    print('Cleaned CSV written to', out.resolve())


In [11]:
# ROBUST STREAMING & FILTER WORKFLOW
# 1. Counts total rows first (reliable).
# 2. Sets probability to hit target successes.
# 3. Keeps ALL failures as "extra" data on top of the target.

import csv
import random
from pathlib import Path

# --- CONFIGURATION ---
# CHANGE THIS PATH to your real 40GB file location
csv_path = Path('/work/shibberu/share/MA384_Data_Mining_Projects_Winter_2025-26/SMART_failure_prediction/S.M.A.R.T/example_data/example_Q3_2025/2025-07-01_reduced (balance test).csv') 
out_path = Path('balanced_dataset_final.csv')

target_successes = 20_000   # We aim for this many WORKING drives
possible_targets = ['failure','failed','target','label','y','is_failed']
random_seed = 42
progress_interval = 500_000

random.seed(random_seed)

# --- HELPER FUNCTION ---
def find_target_idx(header):
    lower = [h.strip().lower() for h in header]
    for t in possible_targets:
        if t in lower:
            return lower.index(t)
    for i, h in enumerate(lower):
        if 'failure' in h or 'fail' == h:
            return i
    return None

if not csv_path.exists():
    raise FileNotFoundError(f'CSV not found: {csv_path}')

# --- PASS 1: COUNT TOTAL ROWS ---
print("Pass 1: Counting total rows...")
with csv_path.open(newline='', encoding='utf-8', errors='replace') as f:
    # Subtract 1 for header
    total_rows = sum(1 for _ in f) - 1

print(f"Total rows found: {total_rows}")

# --- CALCULATION ---
# We ignore failure count here. We just want to downsample the WHOLE file 
# enough to get roughly 'target_successes' amount of normal data.
if total_rows <= target_successes:
    keep_prob = 1.0
else:
    keep_prob = target_successes / total_rows

print(f"Sampling Probability: {keep_prob:.6f}")
print(f"Expected Successes: ~{target_successes}")
print(f"Expected Failures: ALL of them (added on top)")

# --- PASS 2: FILTER & SAVE ---
print("Pass 2: Streaming and filtering...")
kept_failures = 0
kept_successes = 0
rows_seen = 0

with csv_path.open(newline='', encoding='utf-8', errors='replace') as fin, \
     out_path.open('w', newline='', encoding='utf-8') as fout:
    
    reader = csv.reader(fin)
    writer = csv.writer(fout)
    
    # Write Header
    header = next(reader)
    writer.writerow(header)
    
    # Locate Target Column
    target_idx = find_target_idx(header)
    if target_idx is None:
        raise ValueError("Could not find 'failure' column in header.")

    for row in reader:
        rows_seen += 1
        
        # Safety check for ragged rows
        if len(row) <= target_idx:
            continue
            
        val = row[target_idx].strip()
        
        # Check if Failure (1) or Success (0)
        try:
            v = int(float(val))
        except:
            v = 1 if val.lower() in ('true','t','yes','y','1') else 0
            
        if v == 1:
            # RULE 1: KEEP ALL FAILURES
            writer.writerow(row)
            kept_failures += 1
        else:
            # RULE 2: RANDOMLY SAMPLE SUCCESSES
            if random.random() < keep_prob:
                writer.writerow(row)
                kept_successes += 1
        
        if rows_seen % progress_interval == 0:
            print(f"Processed {rows_seen} rows...")

print("--- DONE ---")
print(f"Total Processed: {rows_seen}")
print(f"Failures Kept:   {kept_failures} (100%)")
print(f"Successes Kept:  {kept_successes}")
print(f"Final Dataset:   {kept_failures + kept_successes} rows")
print(f"Saved to: {out_path.resolve()}")


Pass 1: Counting total rows...
Total rows found: 213459
Sampling Probability: 0.093695
Expected Successes: ~20000
Expected Failures: ALL of them (added on top)
Pass 2: Streaming and filtering...
--- DONE ---
Total Processed: 213459
Failures Kept:   5 (100%)
Successes Kept:  20053
Final Dataset:   20058 rows
Saved to: /work/shibberu/share/MA384_Data_Mining_Projects_Winter_2025-26/SMART_failure_prediction/S.M.A.R.T/example_data/example_Q3_2025/balanced_dataset_final.csv
